In [16]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix #Para monstrar un matriz de scater con todos los valores
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [18]:
dataframe = pd.read_csv(r"./SDC2/kidney_disease.csv") 

In [19]:
dataframe.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [20]:
dataframe = dataframe.drop(['id'],1)
dataframe.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [21]:
#dataframe.bgr.str.replace('\t?','')
#dataframe.bu.str.replace('\t?','')
#dataframe.sc.str.replace('\t?','')
#dataframe.sod.str.replace('\t?','')
#dataframe.pot.str.replace('\t?','')
#dataframe.hemo.str.replace('\t?','')

#dataframe['bgr'] = dataframe.bgr.str.replace('?','nan')
#dataframe['bu'] = dataframe.bu.str.replace('?','nan')
#dataframe['sc'] = dataframe.sc.str.replace('?','nan')
#dataframe['sod'] = dataframe.sod.str.replace('?','nan')
#dataframe['pot'] = dataframe.pot.str.replace('?','nan')
#dataframe['hemo'] = dataframe.hemo.str.replace('?','nan')
dataframe['pcv'] = dataframe.pcv.str.replace('?','nan')
dataframe['wc'] = dataframe.wc.str.replace('?','nan')
dataframe['rc'] = dataframe.rc.str.replace('?','nan')
#for index, row in dataframe.iterrows():
#    print(index, row.pcv)

#dataframe.pcv.str.replace('	?','nan')
#dataframe.wc.str.replace('\t?','0')
#dataframe.rc.str.replace('\t?','0')

In [22]:
dataframe.shape

(400, 25)

In [23]:
dataframe.dropna(axis=0, subset=['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane'], inplace = True)

In [24]:
dataframe.shape

(233, 25)

In [25]:
dataframe.loc[dataframe['classification'] == 'ckd\t', 'classification'] = 'ckd'
dataframe.loc[dataframe['classification'] == 'ckd', 'classification'] = 1
dataframe.loc[dataframe['classification'] == 'notckd', 'classification'] = 0
print(dataframe.groupby("classification").size())

classification
0    135
1     98
dtype: int64


In [30]:
X=dataframe.iloc[:,:-1].values
y=dataframe.iloc[:,24].values

#Tratamiento de datos faltantes NaN por medio de sustituirlos por la media
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0) #axis = 0 es por columna, axis=1 es por fila
imputer = imputer.fit(X[:,0:5])
X[:,0:5] = imputer.transform(X[:,0:5])
imputer = imputer.fit(X[:,9:18])
X[:,9:18] = imputer.transform(X[:,9:18])

X.shape

C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


(233, 24)

In [31]:
labelencoder_X = LabelEncoder()
X[:,5] = labelencoder_X.fit_transform(X[:,5])
X[:,6] = labelencoder_X.fit_transform(X[:,6])
X[:,7] = labelencoder_X.fit_transform(X[:,7])
X[:,8] = labelencoder_X.fit_transform(X[:,8])
X[:,18] = labelencoder_X.fit_transform(X[:,18])
X[:,19] = labelencoder_X.fit_transform(X[:,19])
X[:,20] = labelencoder_X.fit_transform(X[:,20])
X[:,21] = labelencoder_X.fit_transform(X[:,21])
X[:,22] = labelencoder_X.fit_transform(X[:,22])
X[:,23] = labelencoder_X.fit_transform(X[:,23])

In [32]:
onehotencoder = OneHotEncoder(categorical_features=[5,6,7,8,18,19,20,21,22,23])
X = onehotencoder.fit_transform(X).toarray()
#for row in X:
#    print(row)

C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [33]:
model = linear_model.LogisticRegression() 
model.fit(X,y)

C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
predictions = model.predict(X) 
print(predictions)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


In [35]:
model.score(X,y)

1.0

In [36]:
validation_size = 0.30
seed = 20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)

In [37]:
prob_predictions = model.predict_proba(X)
print(prob_predictions)

[[1.16706200e-10 1.00000000e+00]
 [3.12919617e-05 9.99968708e-01]
 [1.80125003e-02 9.81987500e-01]
 [8.65245818e-08 9.99999913e-01]
 [4.60205089e-06 9.99995398e-01]
 [6.76830834e-06 9.99993232e-01]
 [3.64455133e-11 1.00000000e+00]
 [1.68918657e-10 1.00000000e+00]
 [3.52932942e-08 9.99999965e-01]
 [7.83509256e-06 9.99992165e-01]
 [1.43482225e-05 9.99985652e-01]
 [4.82745890e-06 9.99995173e-01]
 [9.12644476e-02 9.08735552e-01]
 [5.99526659e-07 9.99999400e-01]
 [8.74186860e-08 9.99999913e-01]
 [3.87430333e-02 9.61256967e-01]
 [1.70771369e-03 9.98292286e-01]
 [1.48347071e-03 9.98516529e-01]
 [4.64447147e-10 1.00000000e+00]
 [1.54682067e-01 8.45317933e-01]
 [1.64178241e-05 9.99983582e-01]
 [1.27808902e-06 9.99998722e-01]
 [3.20347458e-07 9.99999680e-01]
 [3.57167459e-07 9.99999643e-01]
 [2.35767467e-08 9.99999976e-01]
 [1.28106281e-05 9.99987189e-01]
 [9.61332817e-03 9.90386672e-01]
 [6.07053585e-09 9.99999994e-01]
 [1.97287800e-04 9.99802712e-01]
 [3.22806226e-11 1.00000000e+00]
 [6.434615

In [38]:
probs_df = pd.DataFrame(prob_predictions)

In [39]:
X_result = dataframe.reset_index().copy()
X_result["objetivo"] = y.tolist()
X_result["prediccion"] = predictions
X_result = pd.concat([X_result,probs_df],axis=1)
X_result[["objetivo","prediccion",0,1]]

,objetivo,prediccion,0,1
0,1,1,1.167062e-10,1.000000
1,1,1,3.129196e-05,0.999969
2,1,1,1.801250e-02,0.981987
3,1,1,8.652458e-08,1.000000
4,1,1,4.602051e-06,0.999995
...,...,...,...,...
228,0,0,9.957020e-01,0.004298
229,0,0,9.999933e-01,0.000007
230,0,0,9.995450e-01,0.000455
231,0,0,9.998960e-01,0.000104


In [40]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=5, random_state=seed) 
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()) 
print(msg)

Logistic Regression: 0.975568 (0.012221)


C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\rbuenfil\Anaconda3\envs\DataScience\lib\site-packages\sklearn\linear_model\logistic.py:

In [41]:
predictions = model.predict(X_validation) 
print(accuracy_score(Y_validation, predictions))

1.0


In [42]:
print(confusion_matrix(Y_validation, predictions))

[[43  0]
 [ 0 27]]


In [43]:
print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        27

    accuracy                           1.00        70
   macro avg       1.00      1.00      1.00        70
weighted avg       1.00      1.00      1.00        70



In [ ]:
#Analisis con SVN
